In [26]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from metric import get_span
import difflib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
torch.cuda.is_available()

True

In [5]:
# if torch.cuda.is_available():
#     model.to('cuda')
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [6]:
mask_token = tokenizer.mask_token
mask_token_id = tokenizer.convert_tokens_to_ids(mask_token)
log_softmax = torch.nn.LogSoftmax(dim=0)

In [7]:
sent1 = "He is a black professor"
sent2 = "She is a black professor"
sent3 = "He is a while professor"
sent4 = "She is a black professor"

In [8]:
sent1_token_ids = tokenizer.encode(sent1, return_tensors='pt')
sent2_token_ids = tokenizer.encode(sent2, return_tensors='pt')

In [9]:
template1, template2 = get_span(sent1_token_ids[0], sent2_token_ids[0])

In [10]:
def get_log_prob_unigram(masked_token_ids, token_ids, mask_idx, lm):
    """
    Given a sequence of token ids, with one masked token, return the log probability of the masked token.
    """
    
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]

    # get model hidden states
    output = model(masked_token_ids)
    hidden_states = output[0].squeeze(0)
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)

    # we only need log_prob for the MASK tokens
    assert masked_token_ids[0][mask_idx] == mask_id

    hs = hidden_states[mask_idx]
    target_id = token_ids[0][mask_idx]
    log_probs = log_softmax(hs)[target_id]

    return log_probs

In [11]:
lm = {"model": model,
      "tokenizer": tokenizer,
      "mask_token": mask_token,
      "log_softmax": log_softmax,
    }

In [29]:
sent1_log_probs = {}
sent2_log_probs = {}
total_masked_tokens = 0
N = len(template1)

# skipping CLS and SEP tokens, they'll never be masked
for i in range(1, N-1):
    sent1_masked_token_ids = sent1_token_ids.clone().detach()
    sent2_masked_token_ids = sent2_token_ids.clone().detach()

    sent1_masked_token_ids[0][template1[i]] = mask_token_id
    sent2_masked_token_ids[0][template2[i]] = mask_token_id
    total_masked_tokens += 1

    score1 = get_log_prob_unigram(sent1_masked_token_ids, sent1_token_ids, template1[i], lm)
    score2 = get_log_prob_unigram(sent2_masked_token_ids, sent2_token_ids, template2[i], lm)

    sent1_log_probs[template1[i]] = score1.item() # += score1.item()
    sent2_log_probs[template2[i]] = score2.item() # += score2.item()

pair = {
    'sent1_token_ids':sent1_token_ids,
    'sent2_token_ids':sent2_token_ids,
    'sent1_log_probs':sent1_log_probs,
    'sent2_log_probs':sent2_log_probs
}

In [22]:
def get_diff(pair):
    sent1_token_ids = pair['sent1_token_ids'][0]
    sent2_token_ids = pair['sent2_token_ids'][0]
    sent1_log_probs = pair['sent1_log_probs']
    sent2_log_probs = pair['sent2_log_probs']    
    
    seq1 = [str(x) for x in sent1_token_ids.tolist()]
    seq2 = [str(x) for x in sent2_token_ids.tolist()]
    dataframes=[]

    matcher = difflib.SequenceMatcher(None, seq1, seq2)
    template1, template2 = [], []

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        a=[tokenizer.ids_to_tokens[int(x)] for x in seq1[i1:i2]]
        b=[tokenizer.ids_to_tokens[int(x)] for x in seq2[j1:j2]]
        c=[sent1_log_probs.get(x,-np.inf) for x in range(i1,i2)]
        d=[sent2_log_probs.get(x,-np.inf) for x in range(j1,j2)]
        e=[0]*len(a)
        if tag!='equal':
            a=tokenizer.decode([int(x) for x in seq1[i1:i2]])
            b=tokenizer.decode([int(x) for x in seq2[j1:j2]])
            c=[-np.inf]
            d=[-np.inf]
            e=[1]*len(c)
        data = {'token_more':a,
                'token_less':b,
                'score_more':c,
                'score_less':d,
                'bias_tokens':e
               }
        dataframes.append(pd.DataFrame.from_dict(data))
        
    df = pd.concat(dataframes).reset_index(drop=True)
    df = df.assign(prob_more=lambda x: np.exp(x.score_more),
                   prob_less=lambda x: np.exp(x.score_less),
                   prob_diff=lambda x: x.prob_more-x.prob_less,
                   prob_diff_scaled=lambda x: MinMaxScaler().fit_transform(x.prob_diff.values.reshape(-1, 1)))
    return df.drop('bias_tokens',axis=1,inplace=False)

In [27]:
df = get_diff(pair)

In [28]:
df

,token_more,token_less,score_more,score_less,prob_more,prob_less,prob_diff,prob_diff_scaled
0,[CLS],[CLS],-inf,-inf,0.000000e+00,0.000000e+00,0.000000e+00,0.177947
1,he,she,-inf,-inf,0.000000e+00,0.000000e+00,0.000000e+00,0.177947
2,is,is,-0.417109,-0.540704,6.589492e-01,5.823379e-01,7.661133e-02,1.000000
3,a,a,-0.069630,-0.052006,9.327392e-01,9.493230e-01,-1.658374e-02,0.000000
4,black,black,-11.593366,-11.443108,9.227101e-06,1.072313e-05,-1.496028e-06,0.177931
5,professor,professor,-18.602179,-17.574081,8.340201e-09,2.331704e-08,-1.497684e-08,0.177946
6,[SEP],[SEP],-inf,-inf,0.000000e+00,0.000000e+00,0.000000e+00,0.177947
